In [1]:
import sys
sys.path.append('../')
from unit.config import config
from unit.dataset_multiclass import get_dataloaders
from unit.model import ContrastiveLearningModel
import torch
from tqdm.auto import tqdm

In [2]:
train_loader, val_loader  = get_dataloaders(config.BATCH_SIZE,
                                            config.NUM_WORKERS, 
                                            config.MAX_LENGTH,
                                            dataset_path_or_name='imdb')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Conver to Embeddings:   0%|          | 0/25000 [00:00<?, ?it/s]

In [7]:
for i, data in enumerate(train_loader):
    print(f'{i+1}. pair of contrastive')
    input_ids, mask, label = data
    print(input_ids.squeeze().size())
    print(input_ids.squeeze()[:,:10])
    print(mask.squeeze().size())
    print(mask.squeeze()[:,:10])
    print(label.squeeze().size())
    print(label.squeeze())
    print('end')
    if i >= 0:
        break

/ |#                                                  | 0 Elapsed Time: 0:00:00
- | #                                                | 19 Elapsed Time: 0:00:00


1. pair of contrastive
torch.Size([4, 128])
tensor([[  101,  1045,  4384,  2012,  2320,  2008,  2023,  3185,  2428,  2347],
        [  101,  5064,  1010,  2023,  3185,  9020,  2000,  2022,  1999,  5737],
        [  101,  1996,  3819,  4028,  2003, 17910,  2098,  2043,  1037,  2564],
        [  101,  2748,  1010,  1045,  2113,  1045,  1005,  1049,  2028,  1997]])
torch.Size([4, 128])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
torch.Size([4])
tensor([1, 1, 0, 0])
end


In [6]:
for i, data in enumerate(val_loader):
    print(f'{i+1}. pair of contrastive')
    input_ids, mask, label = data
    print(input_ids.squeeze().size())
    print(input_ids.squeeze()[:10])
    print(mask.squeeze().size())
    print(mask.squeeze()[:10])
    print(label.squeeze().size())
    print(label.squeeze())
    print('end')
    if i >= 0:
        break

1. pair of contrastive
torch.Size([4, 128])
tensor([[  101,  1045,  2293, 16596,  1011, 10882,  1998,  2572,  5627,  2000,
          2404,  2039,  2007,  1037,  2843,  1012, 16596,  1011, 10882,  5691,
          1013,  2694,  2024,  2788,  2104, 11263, 25848,  1010,  2104,  1011,
         12315,  1998, 28947,  1012,  1045,  2699,  2000,  2066,  2023,  1010,
          1045,  2428,  2106,  1010,  2021,  2009,  2003,  2000,  2204,  2694,
         16596,  1011, 10882,  2004, 17690,  1019,  2003,  2000,  2732, 10313,
          1006,  1996,  2434,  1007,  1012, 10021,  4013,  3367, 20086,  2015,
          1010, 10036, 19747,  4520,  1010, 25931,  3064, 22580,  1010,  1039,
          2290,  2008,  2987,  1005,  1056,  2674,  1996,  4281,  1010,  1998,
         16267,  2028,  1011,  8789,  3494,  3685,  2022,  9462,  2007,  1037,
          1005, 16596,  1011, 10882,  1005,  4292,  1012,  1006,  1045,  1005,
          1049,  2469,  2045,  2024,  2216,  1997,  2017,  2041,  2045,  2040,
        

In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 加载最佳模型检查点
model = ContrastiveLearningModel(model_name=config.MODEL_NAME)

# 进行推理和评估
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
for i, batch in enumerate(tqdm(train_loader)):
    input_ids, attention_mask, label = [t.squeeze().to(device) for t in batch]
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    print(output)
    print(label)
    loss = torch.nn.functional.cross_entropy(output,label)
    print(loss)
    '''
    pred = torch.argmax(output, dim=1)
    val_preds = pred.cpu()
    val_labels = label.cpu()
    precision = precision_score(val_labels, val_preds, average='micro')
    recall = recall_score(val_labels, val_preds, average='micro')
    f1 = f1_score(val_labels, val_preds, average='micro')
    accuracy = accuracy_score(val_labels, val_preds)

    print('val_precision', precision)
    print('val_recall', recall)
    print('val_f1', f1)
    print('val_accuracy', accuracy)
    '''
    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/ |#                                                  | 0 Elapsed Time: 0:00:00


  0%|          | 0/6250 [00:00<?, ?it/s]

/ |#                                                  | 0 Elapsed Time: 0:00:00
- | #                                                | 20 Elapsed Time: 0:00:00
\ |  #                                               | 25 Elapsed Time: 0:00:00


tensor([[ 0.0088, -0.1258],
        [-0.1019, -0.1434],
        [ 0.1229, -0.2430],
        [ 0.0328,  0.0125]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.7242, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.1608, -0.2034],
        [-0.0213,  0.0165],
        [ 0.2387,  0.0291],
        [ 0.1231, -0.0625]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.7084, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[-1.0900e-01, -2.0336e-01],
        [ 4.8706e-02, -1.7089e-02],
        [-5.7105e-05, -2.3141e-01],
        [-1.1662e-01,  2.6274e-01]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.6612, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.1076, -0.1642],
        [ 0.0751, -0.0298],
        [-0.0769, -0.2263],
        [ 0.0582,  0.0602]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.667

| |   #                                              | 31 Elapsed Time: 0:00:00
/ |    #                                             | 38 Elapsed Time: 0:00:00


...

- |                         #                       | 164 Elapsed Time: 0:00:02
\ |                           #                     | 170 Elapsed Time: 0:00:02


tensor([[ 0.0925, -0.0484],
        [ 0.1172, -0.2793],
        [ 0.1034, -0.1441],
        [-0.0431,  0.0852]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.6488, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.1941, -0.1178],
        [-0.0728, -0.3562],
        [-0.1660, -0.3855],
        [ 0.1804,  0.0826]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.6657, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.1066, -0.3028],
        [ 0.1320, -0.2094],
        [ 0.2466,  0.3388],
        [ 0.2192, -0.2880]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.6682, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[-0.1423,  0.0083],
        [ 0.0425, -0.2610],
        [-0.0986, -0.2032],
        [ 0.0353, -0.2353]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([1, 1, 0, 0], device='cuda:0')
tensor(0.6716, device='cuda:0', grad_fn=<Nll

| |                            #                    | 176 Elapsed Time: 0:00:02
/ |                             #                   | 182 Elapsed Time: 0:00:02


tensor([[-0.1170, -0.1331],
        [ 0.0305,  0.1134],
        [ 0.0951,  0.1754],
        [-0.0202, -0.1046]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.7027, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.1087, -0.2300],
        [ 0.0353, -0.2499],
        [ 0.0278, -0.1014],
        [ 0.2850, -0.1396]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([1, 1, 0, 0], device='cuda:0')
tensor(0.7141, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.1285,  0.0460],
        [ 0.2358,  0.0134],
        [-0.1887, -0.1046],
        [ 0.2155, -0.0650]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.6840, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0.0320, -0.1925],
        [ 0.0616, -0.0064],
        [ 0.0843, -0.1988],
        [-0.2794, -0.2928]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1], device='cuda:0')
tensor(0.6979, device='cuda:0', grad_fn=<Nll

KeyboardInterrupt: 

In [8]:
import numpy as np

# 建立一個形狀為 2x4x128 的矩陣
matrix = np.random.rand(2, 4, 5)

# 獲取矩陣的形狀
shape = matrix.shape

# 計算前兩個維度的乘積
new_shape_first_dim = shape[0] * shape[1]

# 自動設置新的形狀
reshaped_matrix = matrix.reshape(new_shape_first_dim, shape[2])

print("原始矩陣形狀:", matrix.shape)
print("降維後的矩陣形狀:", reshaped_matrix.shape)


原始矩陣形狀: (2, 4, 5)
降維後的矩陣形狀: (8, 5)


In [9]:
matrix

array([[[0.79557872, 0.46039835, 0.56563753, 0.7488221 , 0.46439358],
        [0.7928888 , 0.96784006, 0.08237206, 0.02275389, 0.1669453 ],
        [0.15931366, 0.47206555, 0.57146352, 0.1049263 , 0.31776827],
        [0.36920129, 0.80437327, 0.14996319, 0.16418124, 0.94332088]],

       [[0.31249487, 0.88598467, 0.34534662, 0.46421781, 0.0354572 ],
        [0.0720262 , 0.80569732, 0.10087847, 0.70427672, 0.43372558],
        [0.75418592, 0.34902049, 0.03212869, 0.58542441, 0.61343425],
        [0.86287771, 0.46624563, 0.26669618, 0.88591575, 0.13587397]]])

In [10]:
reshaped_matrix

array([[0.79557872, 0.46039835, 0.56563753, 0.7488221 , 0.46439358],
       [0.7928888 , 0.96784006, 0.08237206, 0.02275389, 0.1669453 ],
       [0.15931366, 0.47206555, 0.57146352, 0.1049263 , 0.31776827],
       [0.36920129, 0.80437327, 0.14996319, 0.16418124, 0.94332088],
       [0.31249487, 0.88598467, 0.34534662, 0.46421781, 0.0354572 ],
       [0.0720262 , 0.80569732, 0.10087847, 0.70427672, 0.43372558],
       [0.75418592, 0.34902049, 0.03212869, 0.58542441, 0.61343425],
       [0.86287771, 0.46624563, 0.26669618, 0.88591575, 0.13587397]])